# Técnicas de fusión

La idea fundamental consiste en aprovechar la información sobre diferentes fuentes, que es recogida por métodos distintos. Por ejemplo, podemos querer combinar texto e imágenes, texto y datos tabulares, series temporales y vídeo, etc.

En definitiva, buscamos combinar en un único modelo de aprendizaje toda la información disponible, no siempre posible de procesar en un único stream, de cara a codificar en el modelo la relación entre las diferentes modalidades de entrada y las etiquetas.

Contrario a lo que pudiera parecer, no existen tantas alternativas. Aquí veremos dos de las fundamentales vías para conseguir dicha fusión de modalidades. En la última sesión veremos una tercera vía.

## Preliminares: modelos de modalidad

Partimos del supuesto de que cada modalidad precisa de su propia arquitectura para ser procesada, o que quizás, hemos trabajado con anterioridad en ambas modalidades por separado, y por tanto contamos con modelos ya pre-entrenados en la tarea de interés.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Concatenate, Input, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential

# Si tuviéramos los modelos guardados en disco, podríamos hacer simplemente:
# tabular_model = keras.models.load_model("tabular_model.h5")
# vision_model = keras.models.load_model("vision_model.h5")


# Definimos un modelo básico para los datos tabulares
def create_tabular_model(input_shape):
    model = Sequential([
        Dense(64, activation="relu", input_shape=input_shape),
        Dense(32, activation="relu"),
        Dense(16, activation="relu", name="tabular_features"),  # Feature vector
        Dense(3, activation="softmax", name="tabular_output")  # Final classification layer
    ], name="TabularModel")
    return model

# Hacemos lo propio para el modelo de vision
def create_vision_model(input_shape):
    model = Sequential([
        Conv2D(32, (3,3), activation="relu", input_shape=input_shape),
        MaxPooling2D(),
        Conv2D(64, (3,3), activation="relu"),
        GlobalAveragePooling2D(),
        Dense(16, activation="relu", name="vision_features"),  # Feature vector
        Dense(3, activation="softmax", name="vision_output")  # Final classification layer
    ], name="VisionModel")
    return model

Nos inventaremos el problema, de tal forma que asumimos que las imágenes tiene tamaños `224 x 224 x 3`, mientras que en la tabla tenemos `10` características por fila:


In [ ]:
# Input shapes
tabular_input_shape = (10,)  # Example: 10 numerical features
vision_input_shape = (224, 224, 3)  # Example: 224x224 RGB images

In [ ]:
tabular_model = create_tabular_model(tabular_input_shape)
tabular_model.summary()

In [ ]:
vision_model = create_vision_model(vision_input_shape)
vision_model.summary()

## Late-fusion

En este tipo de fusión, las modalidades han aprendido ya a predecir la tarea final. Sin embargo, esperamos que donde una modalidad falle, la otra acierte, de tal manera que un pequeño clasificador sobre las moda

In [ ]:
def build_late_fusion_model():
    # Consideramos los mismos inputs que cada modelo por separado
    tabular_input = Input(shape=tabular_input_shape, name="tabular_input")
    vision_input = Input(shape=vision_input_shape, name="vision_input")

    # Obtenemos las predicciones finales para cada modalidad
    tabular_pred = ## Code ##
    vision_pred = ## Code ##

    # Fusión de las probabilidades concatenadas
    merged = ## Code ##
    output = Dense(3, activation="softmax", name="final_output")(merged)

    # Definición final del modelo con nuevo classificador al final
    late_fusion_model = Model(inputs=[tabular_input, vision_input], outputs=output)
    return late_fusion_model

model = build_late_fusion_model()
model.summary()

## Early-fusion

En este caso, en lugar de esperar a que los modelos parciales emitan una decisión final, construiremos un clasificador más sofisticado que no aprenda sobre la base de las probabilidades predichas, sino de la representación interna con la que los modelos de cada modalidad trabajaban.

La idea que subyace es que la extracción de características puede estar bien afinada en ambas modalidades, pero que la parte final, la de decisión, puede en los modelos parciales no ser capaz de resolver la tarea de manera óptima debidas a las interrelaciones entre las modalidades. Es decir, si hay una fuerte relación entre los datos, en lugar de esperar a que cada modelo prediga, elegimos fusionar las representaciones para obtener un embedding mezcla de las modalidades.

En el caso más básico, dicha combinación se implementa como una concatenación de embeddings.

In [ ]:
def build_early_fusion_model():
    # Inputs
    tabular_input = Input(shape=tabular_input_shape, name="tabular_input")
    vision_input = Input(shape=vision_input_shape, name="vision_input")

    tabular_intermediate = Model(
      inputs=tabular_model.input, outputs=tabular_model.get_layer('tabular_features').output)
    vision_intermediate = Model(
      inputs=vision_model.input, outputs=vision_model.get_layer('vision_features').output)

    tabular_features = tabular_intermediate(tabular_input)
    vision_features = vision_intermediate(vision_input)

    # Fusionamos la representación de ambas modalidades
    merged = Concatenate()([tabular_features, vision_features])

    # Añadimos un clasificador, que suele ser más complejo que en late-fusion
    x = Dense(128, activation="relu")(merged)
    x = Dense(64, activation="relu")(x)
    output = Dense(3, activation="softmax", name="final_output")(x)

    early_fusion_model = Model(inputs=[tabular_input, vision_input], outputs=output)
    return early_fusion_model

model = build_early_fusion_model()
model.summary()

Como podemos observar, en ambos casos obtenemos modelos que nos permiten trabajar simultáneamente con ambas modalidades, optimizando la toma de decisiones en escenarios y problemas complejos.

A continuación, os sugiero algunas cuestiones:

- Pueden estos esquemas aplicarse en machine learning tradicional?
- Es mejor late-fusion, o early-fusion?
- Precisamos que los modelos parciales estén ya entrenados en ambas estrategias? Desarrollad la respuesta.
- Podemos entrenar los modelos parciales de nuevos?
- Qué datos (en cuanto a particiones de nuestro dataset) debemos emplear para entrenar los modelos resultantes de la fusión?
- Es necesario que entrenemos un clasificador basados en capas `Dense` cuando utilizamos estas estrategias de fusión?


In [ ]:
from sklearn.linear_model import RidgeClassifier


def build_early_fusion_representation():
    # Inputs
    tabular_input = Input(shape=tabular_input_shape, name="tabular_input")
    vision_input = Input(shape=vision_input_shape, name="vision_input")

    tabular_intermediate = ## Code ##
    vision_intermediate = ## Code ##

    tabular_features = ## Code ##
    vision_features = ## Code ##

    # Fusionamos la representación de ambas modalidades
    merged = Concatenate()([tabular_features, vision_features])

    early_fusion_representation = Model(inputs=[tabular_input, vision_input], outputs=merged)
    return early_fusion_representation

model = build_early_fusion_representation()
model.summary()

Una vez hubiéramos procesado nuestras muestras, tendríamos `N` muestras caracterizadas por embeddings de `32` elementos:

In [ ]:
# asignamos valores aleatorios a los embeddings que lograríamos
X = tf.random.uniform((1000, 32))
y = tf.cast(tf.random.uniform((1000,), minval=0, maxval=1) > 0.5, tf.int32)

model = RidgeClassifier()
_ = model.fit(X, y)
model.score(X, y)